<a href="https://colab.research.google.com/github/hadefpetriza/C23-PC679/blob/main/Project_Capstone_C23_PC679.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification
**Note :**
Jadi Aplikasi dapat mendeteksi kelima item berikut
1.   Sampah Organic
2.   Plastik (Recycle)
3.   Kayu (Recycle) --> data kayu dikit banget
4.   Seng / besi (Recycle)
5.   Non-Recycle(Sampah medis, sampah basah, sampah kaca) --> Baru ada gambar berupa obat pil dan botol kaca




In [ ]:
from keras.models.cloning import Sequential
# Library yang digunakan
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
# import tensorboard
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import VGG19

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# File path for data
train_dataset = "/content/drive/MyDrive/DATASET/TRAIN"
val_dataset = "/content/drive/MyDrive/DATASET/TEST"

In [ ]:
# Add preprocessing layer to the front of VGG
vgg = VGG19(input_shape = [224, 224, 3], weights = "imagenet", include_top = False)

# Set layer.trainable = False (don't train existing weights)
for layer in vgg.layers:
  layer.trainable = False

80134624/80134624 [==============================] - 5s 0us/step


In [ ]:
# Show the data in train_dataset
#jenis = ["Sampah Organic", "Sampah Plastik", "Sampah Kayu", "Sampah Besi", "Sampah Non-Recycle"]
#plt.figure(figsize=(10,10))
#for images, labels in train_dataset.take(1):
#  for i in range(9):
#    ax = plt.subplot(3, 3, i + 1)
#    plt.imshow(images[i].numpy().astype("uint8"))
#    plt.title(jenis[labels[i]])
#    plt.axis("off") 

In [ ]:
# Image Augmentation 
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 40,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(directory = train_dataset,
                                                    batch_size = 32,
                                                    class_mode = 'categorical',
                                                    target_size = (224, 224))

val_datagen = ImageDataGenerator(rescale = 1./255.)
val_generator = val_datagen.flow_from_directory(directory = val_dataset, 
                                                batch_size = 32,
                                                class_mode = 'categorical',
                                                target_size = (224, 224))
  


Found 5313 images belonging to 5 classes.
Found 2 images belonging to 5 classes.


In [ ]:
# Model Architecture

# Flatten the output of the VGG19 model
flatten_layer = tf.keras.layers.Flatten()(vgg.output)

# My layer
dense_layer1 = tf.keras.layers.Dense(512, activation = "relu")(flatten_layer)
output_layer = tf.keras.layers.Dense(5, activation = "softmax")(dense_layer1)

model = tf.keras.models.Model(inputs = vgg.input, outputs = output_layer)

model.compile(optimizer = tf.optimizers.Adam(),
                loss = "categorical_crossentropy",
                metrics = "accuracy")




In [ ]:
# Train Model
history = model.fit(train_generator,
                    epochs = 20,
                    validation_data = val_generator)

Epoch 1/20


KeyboardInterrupt: ignored

In [ ]:
# Show plot train accuracy and val_accuracy
plt.plot(history.history["accuracy"], label = "accuracy")
plt.plot(history.history["val_accuracy"], label = "val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0.5, 1])
plt.legend(loc = "lower right")

In [ ]:
# Save file training (TensorFlow Lite)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model_classification.tflite", 'wb') as f:
  f.write(tflite_model)